In [11]:
##Single Transfer

from web3 import Web3

w3 = Web3(Web3.HTTPProvider('https://testnet-rpc.superposition.so/'))

if not w3.is_connected():
    print("Failed to connect to the network")
    exit()



my_address = 'Your_Address'
private_key = 'Your_Private_Key'

token_address = 'Token_Address_that_you_deployed'

token_contract = w3.eth.contract(address=token_address, abi=[
    {
        "constant": True,
        "inputs": [
            {"name": "_owner", "type": "address"}
        ],
        "name": "balanceOf",
        "outputs": [
            {"name": "", "type": "uint256"}
        ],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [
            {"name": "_to", "type": "address"},
            {"name": "_value", "type": "uint256"}
        ],
        "name": "transfer",
        "outputs": [
            {"name": "", "type": "bool"}
        ],
        "payable": False,
        "stateMutability": "nonpayable",
        "type": "function"
    }
])

amount = 100* (10 ** 18)

nonce = w3.eth.get_transaction_count(my_address)
tx = token_contract.functions.transfer('0x57b24762fFe1e2285ca1E186E605e1cf6fFE7f80', amount).build_transaction({
    'chainId': 98985,
    'gas': 200000,
    'gasPrice': w3.to_wei('10', 'gwei'),
    'nonce': nonce,
})

signed_tx = w3.eth.account.sign_transaction(tx, private_key)

tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

print(f'Transaction sent\nCheck at: https://testnet-explorer.superposition.so/tx/0x{tx_hash.hex()}')

In [12]:
##Multiple Transfer from CSV File

import pandas as pd
from web3 import Web3
import time  # Import the time module for adding delay

# Connect to the Web3 provider
w3 = Web3(Web3.HTTPProvider('https://testnet-rpc.superposition.so/'))

if not w3.is_connected():
    print("Failed to connect to the network")
    exit()

# Your wallet details (convert to checksum address)
my_address = w3.to_checksum_address('Your_Address')
private_key = 'Your_Private_Key_DYOR'

# Token contract address (convert to checksum address)
token_address = w3.to_checksum_address('Token_Address_that_you_deployed')
token_contract = w3.eth.contract(address=token_address, abi=[
    {
        "constant": True,
        "inputs": [
            {"name": "_owner", "type": "address"}
        ],
        "name": "balanceOf",
        "outputs": [
            {"name": "", "type": "uint256"}
        ],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [
            {"name": "_to", "type": "address"},
            {"name": "_value", "type": "uint256"}
        ],
        "name": "transfer",
        "outputs": [
            {"name": "", "type": "bool"}
        ],
        "payable": False,
        "stateMutability": "nonpayable",
        "type": "function"
    }
])

# Define the amount to send (in tokens, converted to the correct decimals)
amount = 1 * (10 ** 18)  # 1 tokens (converted to uint256 by multiplying by 10^18)

# CSV file containing addresses (adjust file path as needed)
csv_file = 'addresses.csv'  # Example CSV with just an 'address' column

# Read the CSV file
df = pd.read_csv(csv_file)

# Loop through each row in the CSV file
for index, row in df.iterrows():
    to_address = w3.to_checksum_address(row['address'])  # Convert recipient address to checksum format
    
    # Get the nonce for the transaction
    nonce = w3.eth.get_transaction_count(my_address)

    # Build the transaction
    tx = token_contract.functions.transfer(to_address, amount).build_transaction({
        'chainId': 98985,
        'gas': 200000,
        'gasPrice': w3.to_wei('10', 'gwei'),
        'nonce': nonce,
    })

    # Sign the transaction
    signed_tx = w3.eth.account.sign_transaction(tx, private_key)

    # Send the transaction
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

    print(f'Transaction to {to_address} sent.\nCheck at: https://testnet-explorer.superposition.so/tx/{tx_hash.hex()}')

    # Wait for 2 seconds before sending the next transaction
    time.sleep(2)

print("All transactions processed.")
